In [1]:
import os
import requests
cred_url = os.environ["QCLOUD_CONTAINER_INSTANCE_CREDENTIALS_URL"]
r = requests.get(cred_url)
secretId = r.json()["TmpSecretId"]
secretKey = r.json()["TmpSecretKey"]
token = r.json()["Token"]

In [5]:
import os
from qcloud_cos import CosConfig
from qcloud_cos import CosS3Client
from ti.utils import get_temporary_secret_and_token

#### 指定本地文件路径，可根据需要修改。
local_file = "/home/tione/notebook/data/user_creative_ids_train.txt"

#### 用户的存储桶，修改为存放所需数据文件的存储桶，存储桶获取参考腾讯云对象存储
bucket="hc-02-1258788535"

#### 用户的数据，修改为对应的数据文件路径，文件路径获取参考腾讯云对象存储
data_key="user_creative_ids_train.txt"

#### 获取用户临时密钥
secret_id, secret_key, token = get_temporary_secret_and_token()
config = CosConfig(Region=os.environ.get('REGION'), SecretId=secret_id, SecretKey=secret_key, Token=token, Scheme='https')
client = CosS3Client(config)

####  获取文件到本地
response = client.get_object(
    Bucket=bucket,
    Key=data_key,
)
response['Body'].get_stream_to_file(local_file)

In [ ]:
# !pip install gensim

In [17]:
import pandas as pd 
import numpy as np 
import time, gc, os 
from tqdm import tqdm as tqdm  
import logging 
import gensim 
from gensim.models import word2vec   

# 训练cid的300d词向量

In [13]:
data = []
with open("data/user_creative_ids_train.txt", encoding='utf-8') as f:
    for line in tqdm(f):
        line = line.split("\t")[1]
        x1 = [cid for cid in line.split()]
        data.append(x1)
        
with open("data/user_creative_ids_test.txt", encoding='utf-8') as f:
    for line in tqdm(f):
        line = line.split("\t")[1]
        x1 = [cid for cid in line.split()]
        data.append(x1)

3000000it [00:32, 92254.95it/s] 
1000000it [00:10, 97774.82it/s]


In [ ]:
logging.basicConfig(format="%(levelname)s-%(asctime)s:%(message)s", datefmt='%H:%M:%S', level=logging.INFO)

model = word2vec.Word2Vec(sentences=data, size=300, window=12, min_count=1, workers=16, sg=1, iter=10)

model.save('data/word2vec_model/word2vec_cid_300d.model')
# model = word2vec.Word2Vec.load('data/word2vec_model/word2vec_cid_300d.model')

INFO-12:15:19:collecting all words and their counts
INFO-12:15:19:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO-12:15:19:PROGRESS: at sentence #10000, processed 335578 words, keeping 150228 word types
INFO-12:15:19:PROGRESS: at sentence #20000, processed 673775 words, keeping 252881 word types
INFO-12:15:19:PROGRESS: at sentence #30000, processed 1012932 words, keeping 337989 word types
INFO-12:15:19:PROGRESS: at sentence #40000, processed 1344506 words, keeping 411658 word types
INFO-12:15:20:PROGRESS: at sentence #50000, processed 1683247 words, keeping 480886 word types
INFO-12:15:20:PROGRESS: at sentence #60000, processed 2018755 words, keeping 542999 word types
INFO-12:15:20:PROGRESS: at sentence #70000, processed 2356409 words, keeping 601314 word types
INFO-12:15:20:PROGRESS: at sentence #80000, processed 2694811 words, keeping 655662 word types
INFO-12:15:20:PROGRESS: at sentence #90000, processed 3029095 words, keeping 706318 word types
INFO-12:15:20:P

# 训练aderid的300d词向量

In [20]:
ad_df = pd.read_pickle("data/all_ad.pkl")
cid2ader = dict(zip(ad_df["creative_id"], ad_df["advertiser_id"]))

In [23]:
for col in ad_df.columns:
    print(col, ad_df[col].nunique())

creative_id 4445720
ad_id 3812202
product_id 44315
product_category 18
advertiser_id 62965
industry 336


In [25]:
data = []
with open("data/user_creative_ids_train.txt", encoding='utf-8') as f:
    for line in tqdm(f):
        line = line.split("\t")[1]
        x1 = [cid2ader.get(cid, '0') for cid in line.split()]
        data.append(x1)
        
with open("data/user_creative_ids_test.txt", encoding='utf-8') as f:
    for line in tqdm(f):
        line = line.split("\t")[1]
        x1 = [cid2ader.get(cid, '0') for cid in line.split()]
        data.append(x1)
        
logging.basicConfig(format="%(levelname)s-%(asctime)s:%(message)s", datefmt='%H:%M:%S', level=logging.INFO)
model = word2vec.Word2Vec(sentences=data, size=300, window=12, min_count=1, workers=16, sg=1, iter=10)
model.save('data/word2vec_model/word2vec_ader_300d.model')
# model = word2vec.Word2Vec.load('data/word2vec_model/word2vec_ader_300d.model')

3000000it [01:24, 35470.13it/s]
1000000it [00:31, 31814.55it/s]


In [27]:
logging.basicConfig(format="%(levelname)s-%(asctime)s:%(message)s", datefmt='%H:%M:%S', level=logging.INFO)
model = word2vec.Word2Vec(sentences=data, size=300, window=12, min_count=1, workers=16, sg=1, iter=10)
model.save('data/word2vec_model/word2vec_ader_300d.model') 

INFO-14:19:22:collecting all words and their counts
INFO-14:19:22:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO-14:19:22:PROGRESS: at sentence #10000, processed 335578 words, keeping 16253 word types
INFO-14:19:22:PROGRESS: at sentence #20000, processed 673775 words, keeping 20419 word types
INFO-14:19:22:PROGRESS: at sentence #30000, processed 1012932 words, keeping 23071 word types
INFO-14:19:22:PROGRESS: at sentence #40000, processed 1344506 words, keeping 25255 word types
INFO-14:19:22:PROGRESS: at sentence #50000, processed 1683247 words, keeping 27044 word types
INFO-14:19:22:PROGRESS: at sentence #60000, processed 2018755 words, keeping 28464 word types
INFO-14:19:22:PROGRESS: at sentence #70000, processed 2356409 words, keeping 29702 word types
INFO-14:19:23:PROGRESS: at sentence #80000, processed 2694811 words, keeping 30781 word types
INFO-14:19:23:PROGRESS: at sentence #90000, processed 3029095 words, keeping 31842 word types
INFO-14:19:23:PROGRESS: 

# 训练pid的300d词向量

In [28]:
cid2pid = dict(zip(ad_df["creative_id"], ad_df["product_id"]))

data = []
with open("data/user_creative_ids_train.txt", encoding='utf-8') as f:
    for line in tqdm(f):
        line = line.split("\t")[1]
        x1 = [cid2pid.get(cid, '0') for cid in line.split()]
        data.append(x1)
        
with open("data/user_creative_ids_test.txt", encoding='utf-8') as f:
    for line in tqdm(f):
        line = line.split("\t")[1]
        x1 = [cid2pid.get(cid, '0') for cid in line.split()]
        data.append(x1)
        
logging.basicConfig(format="%(levelname)s-%(asctime)s:%(message)s", datefmt='%H:%M:%S', level=logging.INFO)
model = word2vec.Word2Vec(sentences=data, size=300, window=12, min_count=1, workers=16, sg=1, iter=10)
model.save('data/word2vec_model/word2vec_pid_300d.model')
# model = word2vec.Word2Vec.load('data/word2vec_model/word2vec_pid_300d.model')

3000000it [01:24, 35472.67it/s]
1000000it [00:31, 31584.04it/s]
INFO-15:09:30:collecting all words and their counts
INFO-15:09:30:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO-15:09:30:PROGRESS: at sentence #10000, processed 335578 words, keeping 6404 word types
INFO-15:09:30:PROGRESS: at sentence #20000, processed 673775 words, keeping 8709 word types
INFO-15:09:30:PROGRESS: at sentence #30000, processed 1012932 words, keeping 10256 word types
INFO-15:09:30:PROGRESS: at sentence #40000, processed 1344506 words, keeping 11706 word types
INFO-15:09:31:PROGRESS: at sentence #50000, processed 1683247 words, keeping 12879 word types
INFO-15:09:31:PROGRESS: at sentence #60000, processed 2018755 words, keeping 13840 word types
INFO-15:09:31:PROGRESS: at sentence #70000, processed 2356409 words, keeping 14683 word types
INFO-15:09:31:PROGRESS: at sentence #80000, processed 2694811 words, keeping 15454 word types
INFO-15:09:31:PROGRESS: at sentence #90000, processed 30

In [ ]:
cid2adid = dict(zip(ad_df["creative_id"], ad_df["ad_id"]))

data = []
with open("data/user_creative_ids_train.txt", encoding='utf-8') as f:
    for line in tqdm(f):
        line = line.split("\t")[1]
        x1 = [cid2adid.get(cid, '0') for cid in line.split()]
        data.append(x1)
        
with open("data/user_creative_ids_test.txt", encoding='utf-8') as f:
    for line in tqdm(f):
        line = line.split("\t")[1]
        x1 = [cid2adid.get(cid, '0') for cid in line.split()]
        data.append(x1)
        
logging.basicConfig(format="%(levelname)s-%(asctime)s:%(message)s", datefmt='%H:%M:%S', level=logging.INFO)
model = word2vec.Word2Vec(sentences=data, size=300, window=12, min_count=1, workers=16, sg=1, iter=20)
model.save('data/word2vec_model/word2vec_adid_300d.model')
# model = word2vec.Word2Vec.load('data/word2vec_model/word2vec_adid_300d.model')

3000000it [01:25, 34934.71it/s]
1000000it [00:36, 27212.94it/s]
INFO-15:39:29:collecting all words and their counts
INFO-15:39:29:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO-15:39:29:PROGRESS: at sentence #10000, processed 335578 words, keeping 147072 word types
INFO-15:39:29:PROGRESS: at sentence #20000, processed 673775 words, keeping 246581 word types
INFO-15:39:30:PROGRESS: at sentence #30000, processed 1012932 words, keeping 328447 word types
INFO-15:39:30:PROGRESS: at sentence #40000, processed 1344506 words, keeping 398937 word types
INFO-15:39:30:PROGRESS: at sentence #50000, processed 1683247 words, keeping 465034 word types
INFO-15:39:30:PROGRESS: at sentence #60000, processed 2018755 words, keeping 524211 word types
INFO-15:39:30:PROGRESS: at sentence #70000, processed 2356409 words, keeping 579405 word types
INFO-15:39:30:PROGRESS: at sentence #80000, processed 2694811 words, keeping 630648 word types
INFO-15:39:30:PROGRESS: at sentence #90000, pr